<a href="https://colab.research.google.com/github/100472330/100472330_100486483_PRACTICA1/blob/main/Practica1_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PRIMERA PRÁCTICA: PREDICCIÓN DEL ABANDONO (BURNOUT) DE EMPLEADOS

1. Importando librerias necesarias para la realización EDA.python

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

2. Importamos y cargamos los datos necesarios. 

In [5]:
df_available = pd.read_csv("attrition_competition_03.csv")
df_available

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,BusinessTravel,Department,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,6.963504,15,3,4,3.0,4.0,2.0,45,Travel_Rarely,Research & Development,...,5.0,Y,23,8,1,9.0,4,6,0,3
1,7.005464,19,3,3,2.0,3.0,3.0,54,Travel_Rarely,Sales,...,3.0,Y,14,8,0,36.0,5,10,4,7
2,8.334608,11,2,3,1.0,1.0,3.0,29,Travel_Rarely,Research & Development,...,4.0,Y,14,8,1,4.0,5,2,2,0
3,7.521929,6,3,3,4.0,2.0,3.0,25,Travel_Rarely,Research & Development,...,0.0,Y,11,8,1,4.0,3,3,1,2
4,7.040928,16,4,3,2.0,2.0,3.0,21,Travel_Rarely,Sales,...,1.0,Y,14,8,0,1.0,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,7.551838,11,4,3,3.0,4.0,3.0,38,Travel_Rarely,Sales,...,1.0,Y,14,8,0,2.0,3,2,2,1
1466,6.698284,23,3,4,4.0,3.0,3.0,35,Travel_Rarely,Sales,...,2.0,Y,20,8,1,17.0,4,7,0,7
1467,7.717951,9,3,3,4.0,3.0,NaN,36,Non-Travel,Research & Development,...,0.0,Y,14,8,0,4.0,2,3,1,2
1468,6.607642,7,2,4,4.0,4.0,3.0,38,Travel_Rarely,Research & Development,...,0.0,Y,20,8,1,8.0,3,7,0,5


In [3]:
# Mostrando las ultimas 5 filas del dataset
df_available.count()

hrs                        1470
absences                   1470
JobInvolvement             1470
PerformanceRating          1470
EnvironmentSatisfaction    1460
JobSatisfaction            1462
WorkLifeBalance            1453
Age                        1470
BusinessTravel             1470
Department                 1470
DistanceFromHome           1470
Education                  1470
EducationField             1470
EmployeeCount              1470
EmployeeID                 1470
Gender                     1470
JobLevel                   1470
JobRole                    1470
MaritalStatus              1470
MonthlyIncome              1470
NumCompaniesWorked         1464
Over18                     1470
PercentSalaryHike          1470
StandardHours              1470
StockOptionLevel           1470
TotalWorkingYears          1467
TrainingTimesLastYear      1470
YearsAtCompany             1470
YearsSinceLastPromotion    1470
YearsWithCurrManager       1470
dtype: int64

Verificando el tipo de dato que tiene cada columna

In [6]:
df_available.dtypes

hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeID                   int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike            int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromot

1. PRIMERA INSEPECCION GENERAL DE TODOS LOS DATOS

print("\n🔹 Información del dataset:")
print(df_available.info())

print("\n🔹 Valores nulos en cada columna:")
print(df_available.isnull().sum())

print("\n🔹 Distribución de la variable objetivo (Attrition):")
print(df_available["Attrition"].value_counts(normalize=True) * 100)

2. RESUMEN DE LAS ESTADISTICAS DESCRIPTIVAS

In [ ]:
print("\n🔹 Estadísticas de variables numéricas:")
print(df_available.describe())

print("\n🔹 Estadísticas de variables categóricas:")
print(df_available.describe(include=['object']))

In [ ]:
# Check the columns of the DataFrame
print(df_available.columns)

# Ensure the column exists before plotting
col = 'Attrition'
if col in df_available.columns:
    df_available[col].value_counts().plot(kind='bar', ax=ax, color="royalblue", edgecolor="black")
else:
    print(f"Column '{col}' does not exist in the DataFrame")

3. GRÁFICOS Y VISUALIZACIONES DE TODOS LOS DATOS

In [ ]:
# Histograma de variables numéricas
df_available.hist(figsize=(12, 10), bins=30, edgecolor='black')
plt.suptitle("Distribución de Variables Numéricas", fontsize=14)
plt.show()

# Gráficos de barras para variables categóricas
categorical_cols = ['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
       'EducationField', 'EmployeeCount', 'EmployeeID', 'Gender', 'JobLevel',
       'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
       'Over18', 'PercentSalaryHike', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8))

for i, col in enumerate(categorical_cols):
    if i >= len(axes.flatten()):
        break
    ax = axes.flatten()[i]
    df_available[col].value_counts().plot(kind='bar', ax=ax, color="royalblue", edgecolor="black")
    ax.set_title(col)
    ax.set_ylabel("Frecuencia")

plt.tight_layout()
plt.show()

# Gráfico de conteo de la variable objetivo
df_available["Attrition"].value_counts().plot(kind='bar', color=['green', 'red'], edgecolor="black")
plt.title("Distribución de la Variable Objetivo (Attrition)")
plt.ylabel("Cantidad de empleados")
plt.xlabel("Attrition (No = 0, Yes = 1)")
plt.xticks(rotation=0)
plt.show()